In [ ]:
import os
from google.colab import drive
import pandas as pd 
import numpy as np
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


In [ ]:


path = "/content/drive/My Drive/data/"
year = "2019"
os.chdir(path)
if os.path.exists(path+"result") == False:
  os.mkdir(path+"result")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#BMI

###drop duplicate based on CASEID

In [ ]:
bmidata = pd.read_sas(path+"MBSAQIP_PUF_"+year+"_SAS/bmifinal.sas7bdat", encoding='utf-8')
bmidata = bmidata.drop_duplicates(subset=['CASEID'])

### Adjust the Units of Height and Weight

In [ ]:
# Adjust all units of height to inch
bmidata.loc[bmidata['HGTUNIT'] == 'cm', 'HGT'] /= 2.54 
bmidata['HGTUNIT'] = 'in'
# Adjust all units of weight closest to surgery to lb
bmidata.loc[bmidata['WGTUNIT_CLOSEST'] == 'kg', 'WGT_CLOSEST'] *= 2.20462
bmidata['WGTUNIT_CLOSEST'] = 'lb'
# Adjust all units of weight highest before surgery to lb
bmidata.loc[bmidata['WGT_HIGH_UNIT_BAR'] == 'kg', 'WGT_HIGH_BAR'] *= 2.20462
bmidata['WGT_HIGH_UNIT_BAR'] = 'lb'
# Adjust all units of weight after surgery to lb
bmidata.loc[bmidata['WGTUNIT_DISCH'] == 'kg', 'WGT_DISCH'] *= 2.20462
bmidata['WGTUNIT_DISCH'] = 'lb'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

### Clean the Data Based on Height and Weight

In [ ]:
# Drop the data with height < 48 inch (4 feet)
index_drop = bmidata[bmidata['HGT'] < 48].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with height > 84 inch (7 feet)
index_drop = bmidata[bmidata['HGT'] > 84].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with weight closest < 100 lb
index_drop = bmidata[bmidata['WGT_CLOSEST'] < 100].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with weight closest > 650 lb
index_drop = bmidata[bmidata['WGT_CLOSEST'] > 650].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with weight highest before surgery < 100 lb
index_drop = bmidata[bmidata['WGT_HIGH_BAR'] < 100].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with weight highest before surgery > 650 lb
index_drop = bmidata[bmidata['WGT_HIGH_BAR'] > 650].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with weight after surgery < 100 lb
index_drop = bmidata[bmidata['WGT_DISCH'] < 100].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with weight after surgery > 650 lb
index_drop = bmidata[bmidata['WGT_DISCH'] > 650].index
bmidata.drop(index_drop, inplace = True)

### Clean the Data Based on BMI and BMI_DISCH

In [ ]:
# Drop the data with BMI < 35
index_drop = bmidata[bmidata['BMI'] < 35].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with BMI_DISCH < 35
index_drop = bmidata[bmidata['BMI_DISCH'] < 35].index
bmidata.drop(index_drop, inplace = True)
# Drop the data with BMI_DISCH > 100
index_drop = bmidata[bmidata['BMI_DISCH'] > 100].index
bmidata.drop(index_drop, inplace = True)

### Drop columns

In [ ]:
bmidata = bmidata.drop(['HGT', 'HGTUNIT', 'WGT_CLOSEST', 'WGTUNIT_CLOSEST', 'WGT_HIGH_BAR', 'WGT_HIGH_UNIT_BAR', 'WGT_DISCH', 'WGTUNIT_DISCH', 'DTBMI', 'BMI_HIGH_BAR'], axis = 1)
bmidata.head()

,CASEID,BMI,BMI_DISCH
0,1119455.0,56.12,51.71
2,1119457.0,49.48,42.41
3,1119600.0,42.74,40.33
4,1119601.0,38.79,37.24
5,1119637.0,51.42,50.99


#Main Data

In [ ]:
maindata = pd.read_sas(path+"MBSAQIP_PUF_"+year+"_SAS/mainfinal.sas7bdat")
maindata.shape
maindata.head()

/usr/local/lib/python3.7/dist-packages/pandas/io/sas/sas7bdat.py:809: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._string_chunk[js, :]
/usr/local/lib/python3.7/dist-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")


,CASEID,SEX,AGE,ageGT80,race_PUF,hispanic,CPT,CPTUNLISTED_REVCONV,CPTUNLISTED_GASBYPASS,CPTUNLISTED_GASPLICATION,...,DTPROGRSRENALINSUF,DTACTERENALFAILURE,DTCVA,DTCOMA24HOURS,DTPREIFNRVINJ,DTCARDIACARRESTCPR,DTMYOCARDIALINFR,DTTRANSFINTOPPSTOP,DTVEINTHROMBREQTER,DTCDIFF
0,1119455.0,b'Female',22.29,b'No',b'Black or African American',b'No',b'43644',0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1119456.0,b'Female',48.61,b'No',b'White',b'No',b'43644',0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1119457.0,b'Female',48.27,b'No',b'Black or African American',b'No',b'43775',0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1119600.0,b'Female',44.68,b'No',b'Unknown/Not Reported',b'No',b'43775',0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1119601.0,b'Female',40.78,b'No',b'White',b'Yes',b'43775',0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###Adjust all the unit

In [ ]:
# Adjust all units of height to inch
maindata.loc[maindata['HGTUNIT'] == 'cm', 'HGT'] /= 2.54 
maindata['HGTUNIT'] = 'in'
# Adjust all units of weight closest to surgery to lb
maindata.loc[maindata['WGTUNIT_CLOSEST'] == 'kg', 'WGT_CLOSEST'] *= 2.20462
maindata['WGTUNIT_CLOSEST'] = 'lb'
# Adjust all units of weight highest before surgery to lb
maindata.loc[maindata['WGT_HIGH_UNIT_BAR'] == 'kg', 'WGT_HIGH_BAR'] *= 2.20462
maindata['WGT_HIGH_UNIT_BAR'] = 'lb'
# Adjust all units of bougie
maindata['BOUGIE_SIZE'] = maindata['BOUGIE_SIZE'] / maindata['BOUGIE_SIZE_UNITS']

###Drop selected columns

In [ ]:
#drop the columns with almost all missing 
if year == "2015":
  maindata = maindata.drop(columns=['CPTUNLISTED_140101', 'WGT_HIGH_BAR','WGT_HIGH_UNIT_BAR','BMI_HIGH_BAR','IVC_TIMING','CONVERSION','BOUGIE_SIZE','BOUGIE_SIZE_UNITS','PYLORUS_DISTANCE','DTDEATH_OP','DTASSESS_OP','BLEEDING_UNITS','DEATH_CAUSE_BAR','DEATHREVIEW_BAR'])
else:
  maindata = maindata.drop(columns=['CPTUNLISTED_140101', 'WGT_HIGH_BAR','WGT_HIGH_UNIT_BAR','BMI_HIGH_BAR','IVC_TIMING','CONVERSION','BOUGIE_SIZE','BOUGIE_SIZE_UNITS','PYLORUS_DISTANCE','DTDEATH_OP','DTASSESS_OP','ANESTYPE','BLEEDING_UNITS','CDIFF_TSTTYPE','CDIFF_TSTRSLT','CDIFF_TRTMT','CDIFF_DIA','DEHYD_NUM_TRTMTS','EMERG_NUM_TIMES','DEATH_CAUSE_BAR','DEATHREVIEW_BAR','DTPOSTOPSUPERFICIALINCISIONALSSI','DTPOSTOPDEEPINCISIONALSSI','DTPOSTOPORGANSPACESSI','DTPOSTOPVENTILATOR','DTPOSTOPPNEUMONIA','DTPOSTOPSEPSIS','DTPOSTOPSEPTICSHOCK','DTPOSTOPUTI','DTWOUNDDISRUPTION','DTUNPLINTUBATION','DTPULMONARYEMBOLSM','DTPROGRSRENALINSUF','DTACTERENALFAILURE','DTCVA','DTCOMA24HOURS','DTPREIFNRVINJ','DTCARDIACARRESTCPR','DTMYOCARDIALINFR','DTTRANSFINTOPPSTOP','DTVEINTHROMBREQTER','DTCDIFF'])
#maindata = maindata.drop(columns=['STAPLE_LINE_REINFORCEMENT','OVERSEW']) which misses 7w of 20w, but recommend by med student
#drop the columns with many missing 
if (year == "2018") or (year == "2017") or (year == "2019"):
  maindata = maindata.drop(columns=['ALBUMIN','DPRALBUM','HCT','DPRHCT','CREATININE','DPRCREAT','HEMO','DPRHEMO']) # for 2018
if (year == "2016") or (year == "2015"):
    maindata = maindata.drop(columns=['ALBUMIN','DPRALBUM','HCT','DPRHCT']) # for 2018


#drop the colums with these meaning 
#OTHCPT - Other Procedure
#CONCPT - Concurrent Procedure
#DTOP Num Days to operation from hospital admit Days to initial bariatric or metabolic surgery operation date from hospital admission date
#DTDISCH_OP Num Days to discharge from operation Days to hospital discharge from initial bariatric or metabolic surgery date
#DTDISCH_ADMIT Num Days to discharge from hospital admit Days to hospital discharge from hospital admission date
#DTDEATH_OP Num Days to death from operation Days to death from initial bariatric surgery operation date
#DTASSESS_OP Num Days to Post-Op assessment from operation Days to post-operative assessment from initial bariatric surgery operation date
#OPYEAR - Year of initial bariatric or metabolic surgery
#HGTUNIT - Unit
#WGTUNIT_CLOSEST - Unit
#WGT_HIGH_UNIT_BAR - Unit
#BOUGIE_SIZE_UNITS - Unit
maindata = maindata.drop(columns=['OTHCPT1','OTHCPT2','OTHCPT3','OTHCPT4','OTHCPT5','OTHCPT6','OTHCPT7','OTHCPT8','OTHCPT9','OTHCPT10','CONCPT1','CONCPT2','CONCPT3','CONCPT4','CONCPT5','CONCPT6','CONCPT7','CONCPT8','CONCPT9','CONCPT10','DTOP','DTDISCH_OP','DTDISCH_ADMIT','OPYEAR','HGTUNIT','WGTUNIT_CLOSEST'])

#drop based on selection
if (year == "2018") or (year == "2017") or (year == "2019"):
  maindata = maindata.drop(columns=['ageGT80','hispanic','CPTUNLISTED_GASBYPASS','CPTUNLISTED_ENDOTHER','CPTUNLISTED_OTHER','CPTUNLISTED_BALLOON','CPTUNLISTED_VBTHERAPY','STAPLING_PROC','HGT','WGT_CLOSEST','PREV_ORGTRANS','ASSISTANT_TRAINING_LEVEL','PRIORITY','SURGICAL_APPROACH','APPROACH_CONVERTED','DRAIN_PLACED','ANASTOMOSIS_CHECKED','OPLENGTH','SURGSPECIALTY_BAR','ACTERENALFAILURE','CARDIACARRESTCPR','COMA24HOURS','CVA','POSTOPDEEPINCISIONALSSI','DSSIPATOS','MYOCARDIALINFR','POSTOPVENTILATOR','VENTPATOS','POSTOPORGANSPACESSI','OSSIPATOS','POSTOPPNEUMONIA','PNAPATOS','PREIFNRVINJ','PROGRSRENALINSUF','PULMONARYEMBOLSM','POSTOPSEPSIS','SEPSISPATOS','POSTOPSEPTICSHOCK','SEPSHOCKPATOS','POSTOPSUPERFICIALINCISIONALSSI','SSSIPATOS','TRANSFINTOPPSTOP','UNPLINTUBATION','POSTOPUTI','UTIPATOS','VEINTHROMBREQTER','WOUNDDISRUPTION','UNPLANNEDADMISSIONICU30','CDIFF','DEHYD_TRTMT_OUT','EMERG_VISIT_OUT','DISCHARGE_DESTINATION','FOLLOW_30DAYS_BAR','EXAM_PERFORMED_PERSON_BAR','ANTICOAGULATION_INITIATED_BAR','INCISIONAL_HERNIA_NOTED_BAR','DRAIN_PRESENT_30DAY_BAR'])
if (year == "2016"):
  maindata = maindata.drop(columns=['ageGT80','hispanic','CPTUNLISTED_GASBYPASS','CPTUNLISTED_ENDOTHER','CPTUNLISTED_OTHER','CPTUNLISTED_BALLOON','STAPLING_PROC','HGT','WGT_CLOSEST','ASSISTANT_TRAINING_LEVEL','PRIORITY','SURGICAL_APPROACH','APPROACH_CONVERTED','DRAIN_PLACED','ANASTOMOSIS_CHECKED','OPLENGTH','SURGSPECIALTY_BAR','ACTERENALFAILURE','CARDIACARRESTCPR','COMA24HOURS','CVA','POSTOPDEEPINCISIONALSSI','DSSIPATOS','MYOCARDIALINFR','POSTOPVENTILATOR','VENTPATOS','POSTOPORGANSPACESSI','OSSIPATOS','POSTOPPNEUMONIA','PNAPATOS','PREIFNRVINJ','PROGRSRENALINSUF','PULMONARYEMBOLSM','POSTOPSEPSIS','SEPSISPATOS','POSTOPSEPTICSHOCK','SEPSHOCKPATOS','POSTOPSUPERFICIALINCISIONALSSI','SSSIPATOS','TRANSFINTOPPSTOP','UNPLINTUBATION','POSTOPUTI','UTIPATOS','VEINTHROMBREQTER','WOUNDDISRUPTION','UNPLANNEDADMISSIONICU30','CDIFF','DEHYD_TRTMT_OUT','EMERG_VISIT_OUT','DISCHARGE_DESTINATION','FOLLOW_30DAYS_BAR','EXAM_PERFORMED_PERSON_BAR','ANTICOAGULATION_INITIATED_BAR','INCISIONAL_HERNIA_NOTED_BAR','DRAIN_PRESENT_30DAY_BAR'])
if (year == "2015"):
  maindata = maindata.drop(columns=['agegt80','hispanic','CPTUNLISTED_GASBYPASS','CPTUNLISTED_ENDOTHER','CPTUNLISTED_OTHER','STAPLING_PROC','HGT','WGT_CLOSEST','ASSISTANT_TRAINING_LEVEL','PRIORITY','SURGICAL_APPROACH','APPROACH_CONVERTED','DRAIN_PLACED','ANASTOMOSIS_CHECKED','OPLENGTH','ACTERENALFAILURE','CARDIACARRESTCPR','COMA24HOURS','CVA','POSTOPDEEPINCISIONALSSI','DSSIPATOS','MYOCARDIALINFR','POSTOPVENTILATOR','VENTPATOS','POSTOPORGANSPACESSI','OSSIPATOS','POSTOPPNEUMONIA','PNAPATOS','PREIFNRVINJ','PROGRSRENALINSUF','PULMONARYEMBOLSM','POSTOPSEPSIS','SEPSISPATOS','POSTOPSEPTICSHOCK','SEPSHOCKPATOS','POSTOPSUPERFICIALINCISIONALSSI','SSSIPATOS','TRANSFINTOPPSTOP','UNPLINTUBATION','POSTOPUTI','UTIPATOS','VEINTHROMBREQTER','WOUNDDISRUPTION','UNPLANNEDADMISSIONICU30','DISCHARGE_DESTINATION','FOLLOW_30DAYS_BAR','EXAM_PERFORMED_PERSON_BAR','ANTICOAGULATION_INITIATED_BAR','INCISIONAL_HERNIA_NOTED_BAR','DRAIN_PRESENT_30DAY_BAR'])
#drop the columns with 70000 missing of 200000 - these two facotrs only occur in Sleeve
maindata = maindata.drop(columns=['STAPLE_LINE_REINFORCEMENT','OVERSEW'])

###Change CPT into 4 Categories - RYGB, Sleeve, Band, BPD/DS

In [ ]:
RYGB = np.empty(len(maindata))
Sleeve = np.empty(len(maindata))
Band = np.empty(len(maindata))
BPDDS = np.empty(len(maindata))
SADIS = np.empty(len(maindata))
Death = np.empty(len(maindata))
Unknown = np.empty(len(maindata))

CPT = maindata.CPT
DEATH30 = maindata.DEATH_RELATED_BAR
FUNSTATPRESURG = maindata.FUNSTATPRESURG


In [ ]:
for index in range(len(maindata)):
    operation_number = CPT[index]

    #Replace DEATH30 and DEATH_RELATED_BAR two colomns with one DEATH
    if DEATH30[index] == b'Yes':
      Death[index] = 1
    else:
      Death[index] = 0
    #Find the unknown in FUNSTATPRESURG and drop them
    if FUNSTATPRESURG[index] == b'Unknown':
      Unknown[index] = np.nan
    else:
      Unknown[index] = 1
    #Categorize all CPT
    if (operation_number == b'43644')\
    | (operation_number == b'43645')\
    | (operation_number == b'43846')\
    | (operation_number == b'43847'):
        RYGB[index] = 1
        Sleeve[index] = np.nan
        Band[index] = np.nan
        BPDDS[index] = np.nan
        SADIS[index] = np.nan

    elif (operation_number == b'43775')\
    | (operation_number == b'43842'):
        RYGB[index] = np.nan
        Sleeve[index] = 1
        Band[index] = np.nan
        BPDDS[index] = np.nan
        SADIS[index] = np.nan
    
    elif operation_number == b'43770':
        RYGB[index] = np.nan
        Sleeve[index] = np.nan
        Band[index] = 1
        BPDDS[index] = np.nan
        SADIS[index] = np.nan

    elif operation_number == b'43845':
        RYGB[index] = np.nan
        Sleeve[index] = np.nan
        Band[index] = np.nan
        BPDDS[index] = 1
        SADIS[index] = np.nan

    elif (operation_number == b'43659')\
    | (operation_number == b'43999'):
        RYGB[index] = np.nan
        Sleeve[index] = np.nan
        Band[index] = np.nan
        BPDDS[index] = np.nan
        SADIS[index] = 1

    elif (operation_number != b'43644')\
    & (operation_number != b'43645')\
    & (operation_number != b'43846')\
    & (operation_number != b'43847')\
    & (operation_number != b'43775')\
    & (operation_number != b'43842')\
    & (operation_number != b'43770')\
    & (operation_number != b'43845')\
    & (operation_number != b'43659')\
    & (operation_number != b'43999'): 
        RYGB[index] = np.nan
        Sleeve[index] = np.nan
        Band[index] = np.nan
        BPDDS[index] = np.nan
        SADIS[index] = np.nan


In [ ]:
RYGB_df = pd.DataFrame(RYGB,columns = ['RYGB'])
Sleeve_df = pd.DataFrame(Sleeve,columns = ['Sleeve'])
Band_df = pd.DataFrame(Band,columns = ['Band'])
BPDDS_df = pd.DataFrame(BPDDS,columns = ['BPD/DS'])
SADIS_df = pd.DataFrame(SADIS,columns = ['SADI-S'])
Death_df = pd.DataFrame(Death,columns = ['DEATH'])
Unknown_df =  pd.DataFrame(Unknown,columns = ['Unknown'])

maindata = pd.concat([maindata,RYGB_df],axis=1)
maindata = pd.concat([maindata,Sleeve_df],axis=1)
maindata = pd.concat([maindata,Band_df],axis=1)
maindata = pd.concat([maindata,BPDDS_df],axis=1)
maindata = pd.concat([maindata,SADIS_df],axis=1)
maindata = pd.concat([maindata,Death_df],axis=1)
maindata = pd.concat([maindata,Unknown_df],axis=1)

maindata = maindata.drop(columns=['DEATH30','DEATH_RELATED_BAR'])

In [ ]:
#delete all the unknown in FUNSTATPRESURG 
maindata = maindata.dropna(subset = ['Unknown'])
maindata = maindata.drop(columns=['Unknown','CPT'])


#delete all the operation that are not from the 5 main CPT
maindata = maindata.dropna(subset = ['RYGB','Sleeve','Band','BPD/DS','SADI-S'],how="all")

print(maindata)

           CASEID        SEX    AGE                      race_PUF  \
0       1119455.0  b'Female'  22.29  b'Black or African American'   
1       1119456.0  b'Female'  48.61                      b'White'   
2       1119457.0  b'Female'  48.27  b'Black or African American'   
3       1119600.0  b'Female'  44.68       b'Unknown/Not Reported'   
4       1119601.0  b'Female'  40.78                      b'White'   
...           ...        ...    ...                           ...   
206565  1391002.0  b'Female'  39.65  b'Black or African American'   
206566  1391003.0  b'Female'  63.46                      b'White'   
206567  1391004.0  b'Female'  53.11                      b'White'   
206568  1391005.0  b'Female'  49.40                      b'White'   
206569  1391006.0  b'Female'  55.26                      b'White'   

        CPTUNLISTED_REVCONV  CPTUNLISTED_GASPLICATION    GERD MOBILITY_DEVICE  \
0                       0.0                       0.0   b'No'           b'No'   
1        

###Drop the rows with missing data on selected columns

In [ ]:
maindata.replace(' ', np.nan)
maindata.replace('', np.nan)
maindata = maindata.dropna(subset = ['SEX','AGE','BMI','ASACLASS','race_PUF'])
print(maindata)

           CASEID        SEX    AGE                      race_PUF  \
0       1119455.0  b'Female'  22.29  b'Black or African American'   
1       1119456.0  b'Female'  48.61                      b'White'   
2       1119457.0  b'Female'  48.27  b'Black or African American'   
3       1119600.0  b'Female'  44.68       b'Unknown/Not Reported'   
4       1119601.0  b'Female'  40.78                      b'White'   
...           ...        ...    ...                           ...   
206565  1391002.0  b'Female'  39.65  b'Black or African American'   
206566  1391003.0  b'Female'  63.46                      b'White'   
206567  1391004.0  b'Female'  53.11                      b'White'   
206568  1391005.0  b'Female'  49.40                      b'White'   
206569  1391006.0  b'Female'  55.26                      b'White'   

        CPTUNLISTED_REVCONV  CPTUNLISTED_GASPLICATION    GERD MOBILITY_DEVICE  \
0                       0.0                       0.0   b'No'           b'No'   
1        

###Analysis after clean

In [ ]:

maindata_T = maindata.describe(include = 'all').T
maindata_T["num_nan"] = maindata.isnull().sum(axis = 0)
maindata_T["num_distinct"] = maindata.nunique(axis = 0)
maindata_T.head()
maindata_T.to_csv(path+"result/post_clean_stats_analysis_"+year+".csv")
print(maindata_T)

                                count unique                  top    freq  \
CASEID                       187816.0    NaN                  NaN     NaN   
SEX                            187816      2            b'Female'  151138   
AGE                          187816.0    NaN                  NaN     NaN   
race_PUF                       187816      6             b'White'  130132   
CPTUNLISTED_REVCONV          187816.0    NaN                  NaN     NaN   
CPTUNLISTED_GASPLICATION     187816.0    NaN                  NaN     NaN   
GERD                           187816      2                b'No'  127951   
MOBILITY_DEVICE                187816      2                b'No'  185338   
MI_ALL_HISTORY                 187816      2                b'No'  185489   
PTC                            187816      2                b'No'  184540   
PCARD                          187816      2                b'No'  185938   
HIP                            187816      2                b'No'  101418   

###One hot encoding - Replace char (2~10) categories with number




In [ ]:
# One-hot encoding for all discrete features
for row in maindata_T[maindata_T['num_distinct'] <= 10].index:
    if maindata_T.loc[row]['num_distinct'] == 1:
      continue
    #if the column only contains 2 catogories like sex, drop first colomn
    if maindata_T.loc[row]['num_distinct'] == 2:
      dummies = pd.get_dummies(maindata[row], prefix = row, dummy_na = False, drop_first=True) #drop the first column
      maindata = pd.concat([maindata, dummies], axis = 1)
      maindata.drop([row], axis = 1, inplace = True) # Drop the original feature column
    #if the column more than 2 catogories 
    else:
      dummies = pd.get_dummies(maindata[row], prefix = row, dummy_na = False)
      maindata = pd.concat([maindata, dummies], axis = 1)
      maindata.drop([row], axis = 1, inplace = True) # Drop the original feature column

# Merge mainfinal Datasets and bmi Datasets

In [ ]:
maindata.drop(['BMI'], axis = 1, inplace = True)
# Merge mainfinal and bmi Datasets
main_and_bmi_data = pd.merge(maindata, bmidata, on = 'CASEID', how = 'inner')
#drop the row with missing data
main_and_bmi_data = main_and_bmi_data.dropna(subset = ['BMI','BMI_DISCH'])
# Export into csv
main_and_bmi_data.to_csv('result/main&bmi_data_'+year+'.csv', index = False)

main_and_bmi_data.shape

(155355, 60)

# Concat bmi Datasets

In [ ]:
# Read in Data
main_and_bmi_data_2015 = pd.DataFrame(pd.read_csv('result/main&bmi_data_2015.csv'))
main_and_bmi_data_2016 = pd.DataFrame(pd.read_csv('result/main&bmi_data_2016.csv'))
main_and_bmi_data_2017 = pd.DataFrame(pd.read_csv('result/main&bmi_data_2017.csv'))
main_and_bmi_data_2018 = pd.DataFrame(pd.read_csv('result/main&bmi_data_2018.csv'))
main_and_bmi_data_2019 = pd.DataFrame(pd.read_csv('result/main&bmi_data_2019.csv'))
# Concat of all Datasets
bmifinal_data = pd.concat([main_and_bmi_data_2015, main_and_bmi_data_2016, main_and_bmi_data_2017, main_and_bmi_data_2018, main_and_bmi_data_2019])
print(bmifinal_data.shape)

# Export into csv
bmifinal_data.replace(np.nan,0)
bmifinal_data.to_csv('result/all_bmi&main_data.csv', index = False)

#analysis
bmifinal_data_T = bmifinal_data.describe(include = 'all').T
bmifinal_data_T["num_nan"] = bmifinal_data.isnull().sum(axis = 0)
bmifinal_data_T["num_distinct"] = bmifinal_data.nunique(axis = 0)
print(bmifinal_data_T)

(729482, 60)
                                                       count           mean  \
CASEID                                              729482.0  838223.032127   
AGE                                                 729482.0      44.339344   
RYGB                                                198100.0       1.000000   
Sleeve                                              495917.0       1.000000   
Band                                                 10006.0       1.000000   
BPD/DS                                                9275.0       1.000000   
SADI-S                                               16184.0       1.000000   
SEX_b'Male'                                         729482.0       0.202213   
race_PUF_b'American Indian or Alaska Native'        729482.0       0.004245   
race_PUF_b'Asian'                                   729482.0       0.004631   
race_PUF_b'Black or African American'               729482.0       0.188970   
race_PUF_b'Native Hawaiian or Other Pac

In [ ]:
print(main_and_bmi_data_2015.shape)
print(main_and_bmi_data_2016.shape)
print(main_and_bmi_data_2017.shape)
print(main_and_bmi_data_2018.shape)
print(main_and_bmi_data_2019.shape)


(127638, 60)
(140005, 60)
(151242, 60)
(155242, 60)
(155355, 60)
